# 人脸检测

| 更新时间 | 负责人 | 内容 | 备注 |
| --- | --- | --- | --- |
| 2021年12月2日 | Rui | 初次编写文档 | ---- |
| 2022年12月15日 | Rui | 修改文档的编写方式 | 使用 Jupyter notebook 进行编写文档 |
| 2022年1月18日 | Rui | 修改文档，增加效果图 | 通过测试的平台有 MaixII-Dock，使用的是 MaixPy3 0.4.0 |

## 运行效果

<iframe src="//player.bilibili.com/player.html?aid=720490332&bvid=BV15Q4y1r7HV&cid=405128041&page=1" scrolling="no" border="0" frameborder="no" framespacing="0" allowfullscreen="true"> </iframe>

## 准备

- 在 [MaixHub](https://www.maixhub.com/modelInfo?modelId=24) 上获取模型文件和运行源码（最新版本的系统内置了模型文件 `/home/model/sobel_int8.*`）
- 确认 MaixPy3 版本为 0.4.0 以上
- 使用的硬件为 MaixII-Dock
- 内存卡内是最新版本的镜像系统
- 插卡启动硬件

## 运行开始人脸检测

读取人脸检测模型文件，并部署到 MaixII-Dock 上

In [1]:
class face:
    labels = ["person"]
    anchors = [1.19, 1.98, 2.79, 4.59, 4.53, 8.92, 8.06, 5.29, 10.32, 10.65]
    m = {
        "param": "/home/model/face/yolo2_face_awnn.param",
        "bin": "/home/model/face/yolo2_face_awnn.bin"
    }
    options = {
        "model_type":  "awnn",
        "inputs": {
            "input0": (224, 224, 3)
        },
        "outputs": {
            "output0": (7, 7, (1+4+1)*5)
        },
        "mean": [127.5, 127.5, 127.5],
        "norm": [0.0078125, 0.0078125, 0.0078125],
    }
    def __init__(self):
        from maix import nn
        from maix.nn import decoder
        self.model = nn.load(self.m, opt=self.options)
        self.yolo = decoder.Yolo2(len(self.labels), self.anchors, net_in_size=(224, 224), net_out_size=(7, 7))
        
    def __del__(self):
        del self.model
        del self.yolo
        
global Face
Face = face()
print(Face)

[ rpyc-kernel ]( running at Wed Jan 19 19:29:07 2022 )


运行网络，进行人脸检测

In [ ]:
from maix import camera, display, nn, image
from maix.nn import decoder
labels = ["person"]
while True:
    img = camera.capture().resize(224,224)
    out = Face.model.forward(img.tobytes(), quantize=True, layout="hwc")
    boxes, probs = Face.yolo.run(out, nms=0.3, threshold=0.5, img_size=(224, 224))
    if len(boxes):
        for i, box in enumerate(boxes):
            img.draw_rectangle(box[0], box[1], box[0]+box[2], box[1]+box[3], (255,0,0), 1)
        display.show(img)
    else:
        display.show(img)

[ rpyc-kernel ]( running at Wed Jan 19 19:29:14 2022 )


> 以上代码均可以在 MaixPy3 IDE 上运行，完整的脚本源码可以到[ MaixPy3_scripts ](https://github.com/sipeed/MaixPy3_scripts/blob/master/scripts/nn_find_face.py)仓库中查看